# Data Preprocessing

## Import libraries

In [19]:
import pandas as pd

In [20]:
df = pd.read_csv('data/output/output_dataset.csv')

df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3956,624,16/10/2023,Allemagne,Bluetongue virus,"Moutons, vaches et chèvres",médias,Le bluetongue virus a été détecté en Allemagne...,"Le bluetongue virus, qui affecte principalemen...",Le bluetongue virus a été détecté en Allemagne...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,4007,617,17/10/2023,Allemagne,Bluetongue,Bouvets,médias,Une équipe d'animaux de ferme en Allemagne a é...,L'Allemagne a perdu sa statut de région « sans...,L'Allemagne est actuellement en train de négoc...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4916,768,18/10/2023,Irlande du Nord,Bluetongue et EHD,"Vide de plusieurs espèces, mais principalement...",médias,Le DAERA alerte les agriculteurs à être vigila...,"La maladie affecte principalement les cerfs, m...","Le département de l'Agriculture, de l'Environn..."
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,NaN,inconnu,inconnue,inconnu,médias,NaN,NaN,NaN
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,12/02/2024,"Europe, France, المغرب",Néfropathie aiguë des ruminants,Abovins (bœufs et vaches),médias,Maladie nécrotisante affectant les abovins en ...,La néfropathie aiguë des ruminants est une mal...,La néfropathie aiguë des ruminants est une mal...


In [21]:
print(df['maladie'])

0                                      Bluetongue virus
1                                            Bluetongue
2                                     Bluetongue et EHD
3                                              inconnue
4                       Néfropathie aiguë des ruminants
5                             Taïnos (pour les animaux)
6                                         Nzef el-Wobai
7     ["hamie qla'ee", 'تهاب الجلد الكتيل', 'الantei...
8                                        Thébap du gelo
9                Maladie hémorragique épizootique (MHE)
10               MHE (Maladie hémorragique épizootique)
11                     Maladie hémorragique épizootique
12    Dermatose nodulaire contagieuse (Inf. par le v...
13                                          Tuberculose
14                Dermatose nodulaire contagieuse (DNC)
15                                             inconnue
16                     Maladie hémorragique épizootique
17                                             i

In [22]:
# Dictionnaire de regroupement des maladies
mapping = {
    "Virus de la bleuette": "Fièvre catarrhale ovine",
    "Bluetongue": "Fièvre catarrhale ovine",
    "Bluetongue et EHD": "Maladie hémorragique épizootique",
    "nezf oubawi": "Maladie hémorragique épizootique",
    "Taïon": "Maladie hémorragique épizootique",
    "Nzef oublotaire (Maladie de la perte de sang)": "Maladie hémorragique épizootique",
    "حمى قلاعية": "Fièvre aphteuse",
    "Maladie de tétanos": "Tétanos",
    "Maladie hémorragique épizootique (MHE)": "Maladie hémorragique épizootique",
    "Dermatose nodulaire contagieuse": "Dermatose nodulaire contagieuse",
    "tuberculose bovine": "Tuberculose bovine",
    "Maladie hémorragique épizootique": "Maladie hémorragique épizootique",
    "inconnue": "Maladie inconnue",
    "MHE (Maladie hémorragique épizootique)": "Maladie hémorragique épizootique",
    "IQA": "Maladie inconnue",
    "Fièvre de West Nile": "Fièvre de West Nile",
    "Grippe équine (Inf. par le virus de la)": "Influenza équine",
    "SARS-CoV-2 chez les animaux": "Infection SARS-CoV-2 animale",
    "Influenza équine": "Influenza équine",
    "طاعون المجترات الصغيرة": "Peste des Petits Ruminants",
    "Peste des Petits Ruminants (PPR) et Péripneumonie contagieuse": "Peste des Petits Ruminants",
    "Clavelée et variole caprine": "Clavelée / Variole caprine",
    "Dermatose nodulaire contagieuse (Inf. par le virus)": "Dermatose nodulaire contagieuse",
    "Aethina tumida (Petit coléoptère des ruches)": "Petit coléoptère des ruches",
    "Nécrose hématopoïétique infectieuse": "Nécrose hématopoïétique infectieuse",
    "Maladie de l'alsace verte": "Maladie hémorragique épizootique",
    "Fièvre hémorragique de Crimée-Congo": "Fièvre hémorragique Crimée-Congo",
    "infection à virus du Nil occidental": "Fièvre de West Nile",
    "Wester-Nile Virus": "Fièvre de West Nile",
    "Maladie hémorragique épizootique (Inf. par le virus)": "Maladie hémorragique épizootique",
    "Disease unknown": "Maladie inconnue",
    None: "Maladie inconnue",
    float('nan'): "Maladie inconnue"
}

# Nettoyage : créer la colonne malade_clean
df["maladie"] = df["maladie"].replace(mapping)

# # Vérifier un aperçu
# print(df[["maladie", "maladie_clean"]].head())

In [23]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Bluetongue virus was detected in Europe,Tridge LogoGlobal trading platform Tridge Get ...,en,3956,624,16/10/2023,Allemagne,Bluetongue virus,"Moutons, vaches et chèvres",médias,Le bluetongue virus a été détecté en Allemagne...,"Le bluetongue virus, qui affecte principalemen...",Le bluetongue virus a été détecté en Allemagne...
1,code152,https://lc.cx/sXWRhi,Bluetongue officially confirmed in Germany,Tridge LogoGlobal trading platform Tridge Get ...,en,4007,617,17/10/2023,Allemagne,Fièvre catarrhale ovine,Bouvets,médias,Une équipe d'animaux de ferme en Allemagne a é...,L'Allemagne a perdu sa statut de région « sans...,L'Allemagne est actuellement en train de négoc...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,DAERA urges farmers to be vigilant for blueton...,Latest News Beef Dairy Sheep Tillage Machinery...,en,4916,768,18/10/2023,Irlande du Nord,Maladie hémorragique épizootique,"Vide de plusieurs espèces, mais principalement...",médias,Le DAERA alerte les agriculteurs à être vigila...,"La maladie affecte principalement les cerfs, m...","Le département de l'Agriculture, de l'Environn..."
3,code154,https://lc.cx/fnYicq,Erreur,Erreur: Message: unknown error: net::ERR_NAME_...,en,566,42,NaN,inconnu,Maladie inconnue,inconnu,médias,NaN,NaN,NaN
4,code155,https://lc.cx/V_Nr3n,اقتصاد,بعد موجة بق الفراش، ترزح العديد من الدول الأور...,ar,1169,191,12/02/2024,"Europe, France, المغرب",Néfropathie aiguë des ruminants,Abovins (bœufs et vaches),médias,Maladie nécrotisante affectant les abovins en ...,La néfropathie aiguë des ruminants est une mal...,La néfropathie aiguë des ruminants est une mal...


In [24]:
df.columns

Index(['code', 'url', 'titre', 'contenu', 'langue', 'nb_caracteres', 'nb_mots',
       'date_publication', 'lieu', 'maladie', 'animal', 'source_publication',
       'resume_50_mots', 'resume_100_mots', 'resume_150_mots'],
      dtype='object')

In [31]:
from deep_translator import GoogleTranslator
from googletrans import Translator
from textblob import TextBlob
import pandas as pd
import re

# Colonnes à traduire
cols = ['titre', 'contenu', 'lieu', 'maladie', 'animal',
        'resume_50_mots', 'resume_100_mots', 'resume_150_mots']

dt = GoogleTranslator(source="auto", target="fr")
gt = Translator()

url_pattern = re.compile(r'^https?://')

def fallback_translate(text):
    try:
        return gt.translate(text, dest='fr').text
    except:
        try:
            return str(TextBlob(text).translate(to='fr'))
        except:
            return text

def translate_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return text
    if url_pattern.match(text.strip()):
        return text

    # 1) deep_translator (rapide)
    try:
        return dt.translate(text)
    except:
        print("⚠️ deep_translator failed → using fallback")
        return fallback_translate(text)

# Traduire uniquement la colonne contenu
df['contenu'] = df['contenu'].apply(translate_text)

print("✅ Traduction terminée. Les codes 178 et 192 ont été traduits correctement.")


⚠️ deep_translator failed → using fallback


C:\Users\rania\AppData\Local\Temp\ipykernel_9316\2113286045.py:18: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  return gt.translate(text, dest='fr').text


⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
⚠️ deep_translator failed → using fallback
✅ Traduction terminée. Les codes 178 et 192 ont été traduits correctement.


In [32]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Le virus de la fièvre catarrhale a été détecté...,Tridge LogoPlateforme de trading mondiale Trid...,en,3956,624,16/10/2023,Allemagne,Virus de la fièvre catarrhale,"Moutons, vaches et chèvres",médias,Le bluetongue virus a été détecté en Allemagne...,"Le bluetongue virus, qui affecte principalemen...",Le bluetongue virus a été détecté en Allemagne...
1,code152,https://lc.cx/sXWRhi,La fièvre catarrhale officiellement confirmée ...,Logo TridgePlateforme commerciale mondiale Tri...,en,4007,617,17/10/2023,Allemagne,Fièvre catarrhale ovine,Bouvets,médias,Une équipe d'animaux de ferme en Allemagne a é...,L'Allemagne a perdu sa statut de région « sans...,L'Allemagne est actuellement en train de négoc...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,Daera exhorte les agriculteurs à être vigilant...,Dernières nouvelles Boeuf Laitier Mouton Machi...,en,4916,768,18/10/2023,Irlande du Nord,Maladie hémorragique épizootique,"Vide de plusieurs espèces, mais principalement...",médias,Le DAERA alerte les agriculteurs à être vigila...,"La maladie affecte principalement les cerfs, m...","Le département de l'Agriculture, de l'Environn..."
3,code154,https://lc.cx/fnYicq,Erreur,Erreur : Message : erreur inconnue : net::ERR_...,en,566,42,NaN,inconnu,Maladie inconnue,inconnu,médias,NaN,NaN,NaN
4,code155,https://lc.cx/V_Nr3n,économie,"Après la vague de punaises de lit, de nombreux...",ar,1169,191,12/02/2024,"Europe, France, Maroc",Néfropathie aiguë des ruminants,Abovins (bœufs et vaches),médias,Maladie nécrotisante affectant les abovins en ...,La néfropathie aiguë des ruminants est une mal...,La néfropathie aiguë des ruminants est une mal...


In [38]:
# Mots-clés à supprimer
mots_inconnus = ["inconnu", "inconnue", "maladie inconnue"]

# Convertir toutes les colonnes en texte pour faciliter la recherche
df = df.astype(str)

# Supprimer toutes les lignes contenant un des mots-clés
df_clean = df[~df.apply(lambda row: row.str.lower().str.contains('|'.join(mots_inconnus)).any(), axis=1)]

# Sauvegarde
df_clean.to_csv("data/output/output_dataset_cleaned.csv", index=False)

print("OK ! Lignes supprimées :", len(df) - len(df_clean))


OK ! Lignes supprimées : 20


In [ ]:
df.head()

,code,url,titre,contenu,langue,nb_caracteres,nb_mots,date_publication,lieu,maladie,animal,source_publication,resume_50_mots,resume_100_mots,resume_150_mots
0,code151,https://lc.cx/nKVbsM,Le virus de la fièvre catarrhale a été détecté...,Tridge LogoPlateforme de trading mondiale Trid...,en,3956,624,16/10/2023,Allemagne,Virus de la fièvre catarrhale,"Moutons, vaches et chèvres",médias,Le bluetongue virus a été détecté en Allemagne...,"Le bluetongue virus, qui affecte principalemen...",Le bluetongue virus a été détecté en Allemagne...
1,code152,https://lc.cx/sXWRhi,La fièvre catarrhale officiellement confirmée ...,Logo TridgePlateforme commerciale mondiale Tri...,en,4007,617,17/10/2023,Allemagne,Fièvre catarrhale ovine,Bouvets,médias,Une équipe d'animaux de ferme en Allemagne a é...,L'Allemagne a perdu sa statut de région « sans...,L'Allemagne est actuellement en train de négoc...
2,code153,https://www.agriland.ie/farming-news/daera-urg...,Daera exhorte les agriculteurs à être vigilant...,Dernières nouvelles Boeuf Laitier Mouton Machi...,en,4916,768,18/10/2023,Irlande du Nord,Maladie hémorragique épizootique,"Vide de plusieurs espèces, mais principalement...",médias,Le DAERA alerte les agriculteurs à être vigila...,"La maladie affecte principalement les cerfs, m...","Le département de l'Agriculture, de l'Environn..."
3,code154,https://lc.cx/fnYicq,Erreur,Erreur : Message : erreur inconnue : net::ERR_...,en,566,42,nan,inconnu,Maladie inconnue,inconnu,médias,nan,nan,nan
4,code155,https://lc.cx/V_Nr3n,économie,"Après la vague de punaises de lit, de nombreux...",ar,1169,191,12/02/2024,"Europe, France, Maroc",Néfropathie aiguë des ruminants,Abovins (bœufs et vaches),médias,Maladie nécrotisante affectant les abovins en ...,La néfropathie aiguë des ruminants est une mal...,La néfropathie aiguë des ruminants est une mal...
